In [ ]:
#import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
task_time =25 #task time (min)
numberofraw = task_time * 60 * 1000

binpath = 'path'
with open(binpath, "rb") as f:
  data = f.read()
  raw_data = np.frombuffer(data, dtype=np.float64)
  raw_data = raw_data.reshape(-1, 5)

if len(raw_data) > numberofraw:
  raw_data = raw_data[:numberofraw]
  #[isosbestic_trigger, green_trigger, flo_data, red_data, red_trigger]
  original_time = [(i+1)/1000 for i in range (len(raw_data))] # to second
  iso_trigger = raw_data[:, 0]
  green_trigger = raw_data[:, 1]
  flo_data = raw_data[:, 2] #405nm, 465nm
  red_data = raw_data[:, 3]
  red_trigger = raw_data[:, 4]

In [ ]:
def logi_func(trigger_data):
  '''
  On:1
  Off:0
  Exclude 2msec at the start and end of excitation light on because of noise
  '''
  logi_list = []
  threshold = 4
  count = 0
    
  trigger_on_start_point = 2
  trigger_on_end_point = 14

  for trigger in trigger_data:
    if trigger > threshold:
      count += 1
      if trigger_on_start_point < count < trigger_on_end_point:
        logi_list.append(1)
      else:
        logi_list.append(0)
    else:
      count = 0
      logi_list.append(0)

  return np.array(logi_list)

In [ ]:
iso_trigger_logiSeq = logi_func(iso_trigger)
green_trigger_logiSeq = logi_func(green_trigger)
red_trigger_logiSeq = logi_func(red_trigger)

#excitation start:1, excitation end:0
#Get start and end indexes
iso_diff = np.diff(iso_trigger_logiSeq)
iso_start_index = [np.where(iso_diff == 1)[0] + 1][0]
iso_end_index = [np.where(iso_diff == -1)[0] + 1][0]

green_diff = np.diff(green_trigger_logiSeq)
green_start_index = [np.where(green_diff == 1)[0] + 1][0]
green_end_index = [np.where(green_diff == -1)[0] + 1][0]

red_diff = np.diff(red_trigger_logiSeq)
red_start_index = [np.where(red_diff == 1)[0] + 1][0]
red_end_index = [np.where(red_diff == -1)[0] + 1][0]

In [ ]:
#Remove bugs
if green_start_index[-1] - green_end_index[-1] > 0:
  green_start_index = green_start_index[:-1]
elif iso_start_index[-1] - iso_end_index[-1] > 0:
  iso_start_index = iso_start_index[:-1]
elif red_start_index[-1] - red_end_index[-1] > 0:
  red_start_index = red_start_index[:-1]

if green_start_index[0] - green_end_index[0] > 0:
  green_end_index = green_end_index[1:]
elif iso_start_index[0] - iso_end_index[0] > 0:
  iso_end_index = iso_end_index[1:]
elif red_start_index[0] - red_end_index[0] > 0:
  red_end_index = red_end_index[1:]

In [ ]:
#Remove bugs
if len(iso_start_index) > len(green_start_index):
  iso_start_index = iso_start_index[1:]
  iso_end_index = iso_end_index[1:]
elif len(iso_start_index) < len(green_start_index):
  green_start_index = green_start_index[1:]
  green_end_index = green_end_index[1:]

if len(iso_start_index) > len(red_start_index):
  iso_start_index = iso_start_index[1:]
  iso_end_index = iso_end_index[1:]
elif len(iso_start_index) < len(red_start_index):
  red_start_index = red_start_index[1:]
  red_end_index = red_end_index[1:]

if len(green_start_index) > len(red_start_index):
  green_start_index = green_start_index[1:]
  green_end_index = green_end_index[1:]
elif len(green_start_index) < len(red_start_index):
  red_start_index = red_start_index[1:]
  red_end_index = red_end_index[1:]

In [ ]:
def data_point_ave_func(start_index, end_index, data):
  '''
  Calculate average of 1 pulse (11 data point/average)
  '''
  data_list = []
  time_list = []
  try:
    for start, end in zip(start_index, end_index):
      data_list.append(np.mean(data[start:end]))
      time_list.append(np.mean(original_time[start:end]))
  except IndexError:
    pass

  return data_list, time_list

In [ ]:
iso_ave, iso_time = data_point_ave_func(iso_start_index, iso_end_index, flo_data)
green_ave, green_time = data_point_ave_func(green_start_index, green_end_index, flo_data)
red_ave, red_time = data_point_ave_func(red_start_index, red_end_index, red_data)

In [ ]:
if len(iso_ave) < len(green_ave):
  green_ave = green_ave[:len(iso_ave)]
  green_time = green_time[:len(iso_ave)]
elif len(iso_ave) > len(green_ave):
  iso_ave = iso_ave[:len(green_ave)]
  iso_time = iso_time[:len(green_ave)]

if len(iso_ave) < len(red_ave):
  red_ave = red_ave[:len(iso_ave)]
  red_time = red_time[:len(iso_ave)]
elif len(iso_ave) > len(red_ave):
  iso_ave = iso_ave[:len(red_ave)]
  iso_time = iso_time[:len(red_ave)]

if len(green_ave) < len(red_ave):
  red_ave = red_ave[:len(green_ave)]
  red_time = red_time[:len(green_ave)]
elif len(green_ave) > len(red_ave):
  green_ave = green_ave[:len(red_ave)]
  green_time = green_time[:len(red_ave)]

In [ ]:
#Calculate moving average
num_conv = np.ones(20)/ 20
movingAve_iso = np.convolve(iso_ave, num_conv, mode="same")
movingAve_green = np.convolve(green_ave, num_conv, mode="same")
movingAve_red = np.convolve(red_ave, num_conv, mode="same")

#Make time list
time_list = [i*0.05 for i in range(1, len(movingAve_iso)+1, 1)]

In [ ]:
movingAve_flot = []
movingAve_flot.append(time_list)
movingAve_flot.append(movingAve_iso)
movingAve_flot.append(movingAve_green)
movingAve_flot.append(movingAve_red)

df = pd.DataFrame(movingAve_flot, index=["time", "iso", "green", "red"]).T
df.to_excel('data.xlsx')